In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = torch.bfloat16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistralai/Mistral-7B-v0.3",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-16 22:46:26 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.5: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.57 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules=["q_proj", "v_proj", "o_proj", "k_proj",
                    "gate_proj", "down_proj", "up_proj", 
                    "embed_tokens","lm_head"],
    lora_alpha=64,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,
    loftq_config=None,
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/home/theo/miniconda3/envs/finetunes/lib/python3.12/site-packages/unsloth/models/_utils.py:752: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filena

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.2.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [5]:
from datasets import Dataset
import json

def load_jsonl_dataset(jsonl_path: str) -> Dataset:
    data_list = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                data_list.append(json.loads(line))
    return Dataset.from_list(data_list)

dataset = load_jsonl_dataset("./processed_data/rawtext/theowriting.jsonl")

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [6]:
for row in dataset[:2]["text"]:
    print("=========================")
    print(row)

I think of all the nicknames I’ve received or given to myself, I like “señor de pastry” the most.

Frankly, even pastry aside, I think it’s an apt descriptor of where I sit socially. It also implies something of a disconnect or unease with “pastry” on the whole. Take that to mean what you will.

It’s not even a real barrier. It’s just the “señor” of it all. That makes me feel like an odd one out.

Because I love pastry, and I love what it stands for — in my mind, at least. It means warmth and welcoming. It means bluntness and no nonsense. And most of all, it means care and regard for the little things.

I don’t even particularly love sweets the way I once did. Not consuming them, at least. But I love constructing them, and I love what they mean to others. I also, somewhat selfishly, love what they have done for me. 

I could wax on, like many others have before me, about the universality of food. About its appeal to the caregiving nature of humans, and its incredible power to transcend

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bf16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=8,

    args=UnslothTrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,

        warmup_ratio=0.05,
        num_train_epochs=10,

        learning_rate=1e-5,
        embedding_learning_rate=1e-5,

        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps=1,
        optim="adamw_torch",
        lr_scheduler_type="cosine",
        weight_decay=0.01,
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

SyntaxError: incomplete input (3134616636.py, line 33)